In [1]:
import torch
import torch.nn.functional as F
from torch.optim.lr_scheduler import StepLR, MultiStepLR
import numpy as np
import torch.nn as nn
from math import *

In [2]:
torch.set_default_tensor_type('torch.DoubleTensor') # 设置浮点类型为 torch.float64

In [3]:
# 定义激活函数: swish(x)
def acti(x):
    return x*torch.sigmoid(x)  

In [4]:
# 定义网络结构
class DeepRitzNet(torch.nn.Module):
    def __init__(self, input_width, layer_width):
        super(DeepRitzNet, self).__init__()
        self.linear_in = torch.nn.Linear(input_width, layer_width)
        self.linear1 = torch.nn.Linear(layer_width, layer_width)
        self.linear2 = torch.nn.Linear(layer_width, layer_width)
        self.linear3 = torch.nn.Linear(layer_width, layer_width)
        self.linear4 = torch.nn.Linear(layer_width, layer_width)
        self.linear5 = torch.nn.Linear(layer_width, layer_width)
        self.linear6 = torch.nn.Linear(layer_width, layer_width)
        self.linear_out = torch.nn.Linear(layer_width, 1)

    def forward(self, x):
        y = self.linear_in(x) # fully connect layer
        y = y + acti(self.linear2(acti(self.linear1(y)))) # residual block 1
        y = y + acti(self.linear4(acti(self.linear3(y)))) # residual block 2
        y = y + acti(self.linear6(acti(self.linear5(y)))) # residual block 3
        output = self.linear_out(y) # fully connect layer
        return output

In [5]:
dimension = 2

In [6]:
# exact solution
def u_ex(x):  
    x_temp = torch.cos(pi*x)
    u_x = (x_temp.sum(1)).reshape([x.size()[0], 1]) # x_temp.sum(1) 按行求和
    return u_x

In [7]:
def f(x):
    x_temp = torch.cos(pi*x)
    f_x = 2*pi**2*(x_temp.sum(1)).reshape([x.size()[0], 1]) # x_temp.sum(1) 按行求和
    return f_x

In [8]:
# Deep Ritz Method
def DRM(x):
    u_hat = model(x)
    # v = torch.ones(u_hat.shape)
    # ux = torch.autograd.grad(outputs = u_hat, inputs = x, grad_outputs = v, create_graph = True)[0]
    
    ux = torch.zeros(x.size()[0], dimension)
    step_size = 0.0001
    for i in range(dimension):
        dx = torch.zeros(x.size()[0], dimension)
        dx[:, i] = torch.ones(x.size()[0])
        ux[:, i] = (model(x+step_size*dx) - model(x-step_size*dx))[:,0]/step_size/2
    uxsq = (torch.sum(ux**2, dim = 1)).reshape([x.size()[0], 1]) # dim = 1 按行求和
    f_temp = f(x)
    part_1 = torch.sum(0.5 * uxsq + 0.5*pi**2*u_hat**2 - f_temp*u_hat)/x.size()[0]
    
    Nb = 100
    xb1 = torch.rand(Nb, dimension)
    xb1[:, 0] = torch.zeros(Nb)
    xb2 = torch.rand(Nb, dimension)
    xb2[:, 0] = torch.ones(Nb)
    xb3 = torch.rand(Nb, dimension)
    xb3[:, 1] = torch.zeros(Nb)
    xb4 = torch.rand(Nb, dimension)
    xb4[:, 1] = torch.ones(Nb)
    dx1 = torch.zeros(xb1.size()[0], dimension)
    dx1[:, 0] = torch.ones(xb1.size()[0])
    uxb1 = ((model(xb1+step_size*dx1) - model(xb1-step_size*dx1))/step_size/2 * (1.0) - 0)**2
    uxb2 = ((model(xb2+step_size*dx1) - model(xb2-step_size*dx1))/step_size/2 * (-1.0) - 0)**2
    dx2 = torch.zeros(xb1.size()[0], dimension)
    dx2[:, 1] = torch.ones(xb1.size()[0])
    uxb3 = ((model(xb3+step_size*dx2) - model(xb3-step_size*dx2))/step_size/2 * (1.0) - 0)**2
    uxb4 = ((model(xb4+step_size*dx2) - model(xb4-step_size*dx2))/step_size/2 * (-1.0) - 0)**2
    part_2 = torch.sum((uxb1 + uxb2 + uxb3 + uxb4)[:,0])/xb1.size()[0]
    lambda1 = 100.0
    return part_1 + lambda1 * part_2 / 4

In [9]:
Data_size = 2000
def Gendata():
    x = torch.rand(Data_size, dimension)
    return x

In [10]:
# 正态分布初始化参数
def initparam(model, sigma):
    for m in model.modules():
        if isinstance(m, nn.Linear):
            m.weight.data.normal_(0, sigma)
    return model

In [11]:
model = DeepRitzNet(dimension, 4)
model = initparam(model, 0.5)

In [12]:
import torch.optim as optim
import torch.nn as nn
import time

In [13]:
def relative_error():
    x = Gendata()
    predict = model(x)
    exact = u_ex(x)
    value = torch.sqrt(torch.sum((predict - exact )**2))/torch.sqrt(torch.sum((exact )**2))
    return value

In [14]:
traintime = 10000
error_save = np.zeros(traintime)
optimizer = optim.Adam(model.parameters())

In [15]:
time_start = time.time()
for i in range(traintime):
    optimizer.zero_grad()
    x = Gendata()
    x.requires_grad = True
    losses = DRM(x)
    losses.backward()
    optimizer.step()
    error = relative_error()
    error_save[i] = float(error)
    
    if i % 50 == 0:
        print("current epoch is: ", i)
        print("current loss is: ", losses.detach())
        print("current relative error is: ", error.detach())
        np.save("DRM_relative_error_2D_Neumann.npy", error_save)
np.save("DRM_relative_error_2D_Neumann.npy", error_save)
time_end = time.time()
print('total time is: ', time_end-time_start, 'seconds')

current epoch is:  0
current loss is:  tensor(17.0821)
current relative error is:  tensor(0.9031)
current epoch is:  50
current loss is:  tensor(5.2000)
current relative error is:  tensor(0.9689)
current epoch is:  100
current loss is:  tensor(1.4700)
current relative error is:  tensor(0.9850)
current epoch is:  150
current loss is:  tensor(0.1536)
current relative error is:  tensor(0.9808)
current epoch is:  200
current loss is:  tensor(-0.0390)
current relative error is:  tensor(0.9824)
current epoch is:  250
current loss is:  tensor(-0.1101)
current relative error is:  tensor(0.9823)
current epoch is:  300
current loss is:  tensor(-0.1346)
current relative error is:  tensor(0.9815)
current epoch is:  350
current loss is:  tensor(-0.1511)
current relative error is:  tensor(0.9822)
current epoch is:  400
current loss is:  tensor(-0.1637)
current relative error is:  tensor(0.9814)
current epoch is:  450
current loss is:  tensor(-0.1702)
current relative error is:  tensor(0.9806)
curren

current epoch is:  4100
current loss is:  tensor(-9.6930)
current relative error is:  tensor(0.0728)
current epoch is:  4150
current loss is:  tensor(-9.8820)
current relative error is:  tensor(0.0660)
current epoch is:  4200
current loss is:  tensor(-9.7434)
current relative error is:  tensor(0.0622)
current epoch is:  4250
current loss is:  tensor(-9.7442)
current relative error is:  tensor(0.0605)
current epoch is:  4300
current loss is:  tensor(-9.9111)
current relative error is:  tensor(0.0581)
current epoch is:  4350
current loss is:  tensor(-9.9935)
current relative error is:  tensor(0.0570)
current epoch is:  4400
current loss is:  tensor(-9.7802)
current relative error is:  tensor(0.0524)
current epoch is:  4450
current loss is:  tensor(-10.4155)
current relative error is:  tensor(0.0509)
current epoch is:  4500
current loss is:  tensor(-9.9908)
current relative error is:  tensor(0.0452)
current epoch is:  4550
current loss is:  tensor(-9.2371)
current relative error is:  tens

current epoch is:  8150
current loss is:  tensor(-9.9539)
current relative error is:  tensor(0.0086)
current epoch is:  8200
current loss is:  tensor(-10.9081)
current relative error is:  tensor(0.0090)
current epoch is:  8250
current loss is:  tensor(-9.4266)
current relative error is:  tensor(0.0123)
current epoch is:  8300
current loss is:  tensor(-9.4523)
current relative error is:  tensor(0.0152)
current epoch is:  8350
current loss is:  tensor(-10.3338)
current relative error is:  tensor(0.0139)
current epoch is:  8400
current loss is:  tensor(-9.2774)
current relative error is:  tensor(0.0093)
current epoch is:  8450
current loss is:  tensor(-10.2367)
current relative error is:  tensor(0.0108)
current epoch is:  8500
current loss is:  tensor(-10.1101)
current relative error is:  tensor(0.0072)
current epoch is:  8550
current loss is:  tensor(-9.5322)
current relative error is:  tensor(0.0096)
current epoch is:  8600
current loss is:  tensor(-9.7183)
current relative error is:  t

In [16]:
x_test = torch.rand(2, 10)
torch.abs(model(x) - u_ex(x))

tensor([[0.0086],
        [0.0015],
        [0.0065],
        ...,
        [0.0056],
        [0.0076],
        [0.0032]], grad_fn=<AbsBackward>)